In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import autokeras as ak
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

import joblib
from sklearn.preprocessing import LabelEncoder

2024-11-02 14:46:51.616161: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 14:46:51.616194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 14:46:51.617104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 14:46:51.622291: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 14:46:52.265022: W tensorflow/compiler/tf2

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-11-02 14:46:53.369911: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-02 14:46:53.403865: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-02 14:46:53.404024: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
ak.__version__

'1.1.0'

In [4]:
tf.__version__

'2.15.1'

In [5]:
df = pd.read_json("./News_Category_Dataset_v3.json", lines=True)

In [6]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   link               209527 non-null  object        
 1   headline           209527 non-null  object        
 2   category           209527 non-null  object        
 3   short_description  209527 non-null  object        
 4   authors            209527 non-null  object        
 5   date               209527 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.6+ MB


In [8]:
df = df.drop(["link", "date"], axis=1)

In [9]:
print(df.duplicated().sum())
df = df.drop_duplicates(subset="short_description")

447


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 187022 entries, 0 to 209526
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   headline           187022 non-null  object
 1   category           187022 non-null  object
 2   short_description  187022 non-null  object
 3   authors            187022 non-null  object
dtypes: object(4)
memory usage: 7.1+ MB


In [11]:
df.isna().sum()

headline             0
category             0
short_description    0
authors              0
dtype: int64

In [13]:
category_count = df["category"].nunique()

In [14]:
encoder = LabelEncoder()
encoder.fit(df["category"])

joblib.dump(encoder, 'label_encoder.pkl')

df["category_code"] = encoder.transform(df["category"])

In [16]:
df["description_len"] = df["short_description"].apply(len)

In [17]:
df['headline_len'] = df['headline'].apply(len)

In [18]:
df = df[df["description_len"] <= 300]

In [19]:
df = df[df["headline_len"] <= 120]

In [20]:
df["authors"] = df["authors"].apply(lambda x: "Unknown" if x == "" else x)

In [21]:
df["news_info"] = df["authors"] + " " +  df["headline"] + " " + df["short_description"]

In [22]:
final_df = df[["news_info", "category_code"]]

In [23]:
del df

In [24]:
final_df = final_df.sample(50000)

In [25]:
final_df

,news_info,category_code
2393,Unknown Assassination Of Haitian President Onl...,40
12711,"TomDispatch, Editorial Partner The Fight Over ...",24
14373,Ron Dicker Shaun White Is Damn Near Perfect In...,28
75178,Carly Ledbetter Stevie Wonder Had The Most Tou...,10
208461,"Kate Hanni, Contributor\nAirline passenger rig...",34
...,...,...
55328,"Kimberly Yam Homeless Learn Coding, Job Skills...",14
134591,"Ellen Hendriksen, PhD, Contributor\nAnxiety wh...",38
116103,"Jon Bird, ContributorGlobal MD for LabStore, Y...",3
56555,Unknown Directory Listing PrEP Providers Acros...,25


In [26]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 2393 to 68381
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   news_info      50000 non-null  object
 1   category_code  50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ MB


In [27]:
X_train, X_tmp, y_train, y_tmp = train_test_split(np.array(final_df.news_info),
                                                  np.array(final_df.category_code),
                                                  test_size=0.2,
                                                  random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_tmp,
                                                  y_tmp,
                                                  test_size=0.5,
                                                  random_state=42)

In [28]:
print('Форма тренировочных данных: ', X_train.shape)
print('Форма тренировочных меток: ', y_train.shape)
print("-"*8)
print('Форма валидационных данных: ', X_val.shape)
print('Форма валидационных меток: ', y_val.shape)
print("-"*8)
print('Форма проверочных данных: ', X_test.shape)
print('Форма проверочных меток: ', y_test.shape)


Форма тренировочных данных:  (40000,)
Форма тренировочных меток:  (40000,)
--------
Форма валидационных данных:  (5000,)
Форма валидационных меток:  (5000,)
--------
Форма проверочных данных:  (5000,)
Форма проверочных меток:  (5000,)


In [29]:
clf = ak.TextClassifier(overwrite=True, max_trials=1, objective='val_accuracy', num_classes=category_count, multi_label=False)

In [30]:
result_training = clf.fit(X_train, y_train, epochs=3, validation_data=(X_val, y_val), 
                        #   batch_size=16
                          )

Trial 1 Complete [00h 01m 01s]
val_accuracy: 0.6430000066757202

Best val_accuracy So Far: 0.6430000066757202
Total elapsed time: 00h 01m 01s
Epoch 1/3
1250/1250 [==============================] - 17s 13ms/step - loss: 2.4589 - accuracy: 0.3651 - val_loss: 1.7867 - val_accuracy: 0.5220
Epoch 2/3
1250/1250 [==============================] - 5s 4ms/step - loss: 1.6470 - accuracy: 0.5682 - val_loss: 1.4442 - val_accuracy: 0.6166
Epoch 3/3
1250/1250 [==============================] - 5s 4ms/step - loss: 1.3309 - accuracy: 0.6392 - val_loss: 1.3467 - val_accuracy: 0.6442
INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


INFO:tensorflow:Assets written to: ./text_classifier/best_model/assets


In [32]:
predicted_y = clf.predict(X_test)

print(clf.evaluate(X_test, y_test))

157/157 [==============================] - 0s 2ms/step - loss: 1.3818 - accuracy: 0.6340
[1.3818411827087402, 0.6340000033378601]


In [33]:
print(classification_report(y_test, predicted_y.astype(int).flatten()))

              precision    recall  f1-score   support

           0       0.33      0.14      0.20        14
           1       0.62      0.76      0.68        21
           2       0.71      0.41      0.52       118
           3       0.47      0.33      0.39       161
           4       1.00      0.05      0.09        21
           5       0.76      0.55      0.64       127
           6       0.37      0.45      0.40        76
           7       0.44      0.15      0.22        27
           8       0.87      0.68      0.76        95
           9       0.00      0.00      0.00        24
          10       0.70      0.70      0.70       422
          11       0.44      0.31      0.36        49
          12       0.67      0.09      0.16        22
          13       0.73      0.74      0.73       171
          14       0.43      0.12      0.19        25
          15       0.19      0.13      0.16        53
          16       0.37      0.62      0.46       110
          17       0.86    

/home/sergey/dev/founder_dirs/hw_26/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sergey/dev/founder_dirs/hw_26/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sergey/dev/founder_dirs/hw_26/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [35]:
parenting = "Brittany Wong Meet The Women Who Say They're 'Canceling Out' Their Dads' Votes What do I mean to my dad when he is willing to put his paycheck before my well-being?"
us_news = "Nick Visser JD Vance Says Anti-Muslim Policies Just Common Sense In Joe Rogan Interview Somehow it`s fundamentally racist to say, well, we don`t want certain people of certain backgrounds to be in the United States of America."

In [39]:
model = clf.export_model()

In [40]:
result_parenting = model.predict(np.array([parenting]))
result_us_news = model.predict(np.array([us_news]))

1/1 [==============================] - 0s 79ms/step


In [41]:
predicted_class_idx_parenting = np.argmax(result_parenting, axis=-1)
predicted_class_idx_us_news = np.argmax(result_us_news, axis=-1)

In [42]:
class_names = encoder.classes_

In [43]:
predicted_category = class_names[predicted_class_idx_parenting[0]]
print(predicted_category)

PARENTING


In [44]:
predicted_category = class_names[predicted_class_idx_us_news[0]]
print(predicted_category)

POLITICS


In [46]:
model.save("model_autokeras.keras")